In [1]:
#Title 

In [2]:
#Deliverable 1 Population Statistics

In [1]:
!pip install wbdata

In [2]:
import wbdata
wbdata.get_topics()

Key '-2914073531465797371' not in persistent cache.
Key '-7058193501900839536' not in persistent cache.
Key '-4378915193300726388' not in persistent cache.
Key '-3761754996952656779' not in persistent cache.
Key '-950751609435370812' not in persistent cache.
Key '-242685482404772510' not in persistent cache.
Key '2568317905112513457' not in persistent cache.
Key '8388231960650947124' not in persistent cache.
Key '-2159427804658413008' not in persistent cache.
Key '8125929092411243436' not in persistent cache.
Key '9147646897019276293' not in persistent cache.
Key '857008287363012477' not in persistent cache.
Key '-1497657898356828032' not in persistent cache.
Key '-2613015786361866379' not in persistent cache.
Key '3188730876195795813' not in persistent cache.
Key '-6139975560448838935' not in persistent cache.
Key '-942699584262088008' not in persistent cache.
Key '2490941882606880699' not in persistent cache.
Key '5755773984605439005' not in persistent cache.
Key '6010011397154764968

  id  value
----  -------------------------------
   1  Agriculture & Rural Development
   2  Aid Effectiveness
   3  Economy & Growth
   4  Education
   5  Energy & Mining
   6  Environment
   7  Financial Sector
   8  Health
   9  Infrastructure
  10  Social Protection & Labor
  11  Poverty
  12  Private Sector
  13  Public Sector
  14  Science & Technology
  15  Social Development
  16  Urban Development
  17  Gender
  18  Millenium development goals
  19  Climate Change
  20  External Debt
  21  Trade

In [3]:
import pandas as pd
import numpy as np

In [4]:
wbdata.get_sources()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Financial Inclusion
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Glob

In [7]:
indicators = wbdata.get_indicators(source=40)
indicators

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.0514        Number of deaths ages 5-14 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1019        Number of deaths ages 10-19 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.0514        Probability of dying at age 5-14 years (per 1,000 children age 5)
SH.DYN.1014        Probability of dying among adolescents ages 1

In [14]:
sexdict = {'Male':'MA', 'Female':'FE'}
def population(year,sex,age_range,place):
        lower, upper = age_range[0], age_range[1]
        df = population_data_frames(sex, age_range, place).reset_index()
        df = df[df['date'] == f'{year}-01-01'] # selecting relevant year
        df.drop(['date', 'country'], axis=1, inplace=True)
        extra = ((upper - (int(df.columns[-1][-4:-2]))) / 5) * (df.iloc[:,-1].to_list()[0]) # controls for age ranges not on bounds, could change to 
                                                                          # year by year subtraction
        final = df.sum(axis=1) - extra
        return f' There were {final.iloc[0]} {sex.lower()}s living in {place} in {year}'

In [15]:
population(1990, 'Male', (10,33), 'CHN')

' There were 246693887.8 males living in CHN in 1990'

In [16]:
def population_data_frames(sex, age_range, place, year = None):
    lower, upper = age_range[0], age_range[1]
    if upper > 80:
        upper = 'UP'
    if upper == lower + 4:
        label = f'SP.POP.{lower:02d}{upper}.{sexdict[sex]}.IN'
        df = wbdata.get_dataframe(label,country=place,parse_dates=True)
    else:
        yrs = [(x, x+4) for x in range(lower, upper, 5)]
        ranges = [f'{x[0]:02d}{x[1]:02d}' for x in yrs]
        var = [f"SP.POP.{x}.{sexdict[sex]}" for x in ranges]
        feed = {x: f"{sex} {ranges[var.index(x)]}" for x in var} # constructs variable dictionary to be fed to wbdata
        
        df = wbdata.get_dataframe(feed,country=place,parse_dates=True).reset_index()
        df['country'] = place
        df.set_index(['country', 'date'], inplace=True)
    return df

In [17]:
population_data_frames(1990, 'Male', (10,33), 'CHN')

TypeError: '>' not supported between instances of 'str' and 'int'

Deliverable 4: Population Pyramids Function

In [18]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np

def population_pyramid(df,year):
    """Plots population pyramid given dataframe from population dataframe function and year."""
    year = f"{year}-01-01"
    age_ranges = [f"{i:02d}{i+4:02d}" for i in range(0, 80, 5)]
    age_ranges.append("80UP")
    layout = go.Layout(barmode='overlay',
                   yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(title='Population'))
    bins = [go.Bar(x = df.loc[year,:].filter(regex="Male").values,
               y = [int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Men',
               marker=dict(color='blue'),
               hoverinfo='skip'
               ),

        go.Bar(x = -df.loc[year,:].filter(regex="Female").values,
               y=[int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Women',
               marker=dict(color='pink'),
               hoverinfo='skip',
               )
        ]
    py.iplot(dict(data=bins, layout=layout))

Deliverable 5: Animated Population Pyramids Function

In [20]:
import plotly.graph_objects as go

def population_pyramid_slider(df):
    """Plots animated population pyramid given dataframe from population dataframe function"""
    years = list(range(1960, 2024))

    age_ranges = [f"{i:02d}{i+4:02d}" for i in range(0, 80, 5)]
    age_ranges.append("80UP")
    
    y_values = [int(s[:2]) for s in age_ranges]

    initial_year = years[0]
    initial_year_str = f"{initial_year}-01-01"

    male_population = df.loc[initial_year_str, :].filter(regex="Male").values
    female_population = df.loc[initial_year_str, :].filter(regex="Female").values

    bars = [
        go.Bar(
            x=male_population,
            y=y_values,
            orientation='h',
            name='Men',
            marker=dict(color='blue'),
            hoverinfo='x+y'
        ),
        go.Bar(
            x=-female_population,
            y=y_values,
            orientation='h',
            name='Women',
            marker=dict(color='pink'),
            hoverinfo='x+y'
        )
    ]

    frames = []
    for year in years:
        year_str = f"{year}-01-01"
        if year_str in df.index:  
            frame_male = df.loc[year_str, :].filter(regex="Male").values
            frame_female = df.loc[year_str, :].filter(regex="Female").values
        else:
            frame_male = [0] * len(y_values)
            frame_female = [0] * len(y_values)

        frames.append(
            go.Frame(
                data=[
                    go.Bar(x=frame_male, y=y_values),
                    go.Bar(x=-frame_female, y=y_values)
                ],
                name=str(year)
            )
        )

    layout = go.Layout(
        title="Population Pyramid Over Time",
        barmode='overlay',
        yaxis=dict(title='Age', range=[0, max(y_values) + 5]),
        xaxis=dict(title='Population', tickformat=",", showgrid=True),
        bargap=0.1,
        template="plotly_white",
        height=600,
        margin=dict(t=150, b=50),
        updatemenus=[{
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 30},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 1.25,
            "yanchor": "top"
        }],
        sliders=[{
            "active": 0,
            "yanchor": "bottom",
            "xanchor": "left",
            "currentvalue": {"prefix": "Year: ", "font": {"size": 20}},
            "pad": {"b": 0, "t": 100},
            "len": 0.9,
            "x": 0.1,
            "y": 1.15,
            "steps": [
                {"args": [[str(year)], {"frame": {"duration": 300, "redraw": True}, "mode": "immediate"}], "label": str(year), "method": "animate"}
                for year in years
            ]
        }]
    )

    fig = go.Figure(data=bars, layout=layout, frames=frames)
    fig.show()